In [101]:
!pip install -q langchain langchain-community langchain_huggingface chromadb

## 1. Use langchain RAG

In [102]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [5]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "u r api"

In [ ]:
# os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [36]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub

# set Korean embedding and llm odel
hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

hf_llm = HuggingFaceHub(
    repo_id="skt/kogpt2-base-v2",
    model_kwargs={"task": "text-generation"} ## question-answering tasK X. text-generation
)

In [ ]:
prompt = f"질문: 5*5? \ n대답:"
response = gemini_model.genterate_content(prompt)

answer = response.candidates[0.contant.parts[0].text
print(answer)

In [37]:
import requests
from langchain.schema import Document
from bs4 import BeautifulSoup

# for Wikipedia documents (EN, KO)

# from langchain_community.document_loaders import WikipediaLoader

# By default, English documents (https://en.wikipedia.org))
# def load_Wiki_docs(query):
#     loader = WikipediaLoader(query=query, load_max_docs=1) # need !pip install wikipedia
#     documents = loader.load()
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200
#     )
#     splits = text_splitter.split_documents(documents)
    
#     return splits


# For Korean query, get results from Korean wikipedia website and crawl and parse results
def load_Korean_wiki_docs(topic):
    url = f"https://ko.wikipedia.org/wiki/{topic}"
    
    response = requests.get(url)
    response.raise_for_status()  # raise Exception when error occurs

    # HTML parsing and extract body contents
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.find('div', {'class': 'mw-parser-output'})  # find div including body contents 
    
    # Extract contents
    paragraphs = content.find_all('p')
    text = "\n".join([p.get_text() for p in paragraphs])  # concat all context in <p> tags 


    
    # convert to Document object (required for LangChain)
    documents = [Document(page_content=text, metadata={"source": url})]
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

In [38]:
def create_vectorstore(splits): 
    vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)
    return vectorstore

In [39]:
topic = "흑백요리사"
# Load wikipedia documents for this topic
splits = load_Korean_wiki_docs(topic) 
# Create vectorstore with this fetched docs
vectorstore = create_vectorstore(splits)

In [40]:
def create_rag_chain(vectorstore):
    prompt_template = """문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: {context}
    질문: {question}
    답변:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}

    # Make context shorter
    # def short_context(context, max_length=300):
    #     return context[:max_length] if len(context) > max_length else context
    
    # class ShortContextRetriever(BaseRetriever):
    #     def __init__(self, retriever):
    #         super().__init__()
    #         self._retriever = retriever
        
    #     def get_relevant_documents(self, query):
    #         docs = self._retriever.get_relevant_documents(query)
    #         for doc in docs:
    #             doc.page_content = short_context(doc.page_content)
    #         return docs
    
    # retriever = ShortContextRetriever(vectorstore.as_retriever())

    qa_chain = RetrievalQA.from_chain_type(
        llm=hf_llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    
    return qa_chain

In [41]:
# create langchang RAG chain
qa_chain = create_rag_chain(vectorstore)

In [42]:
question = "심사위원을 누가 맡았어?"

# result = qa_chain({"query": question})
result = qa_chain.invoke({"query": question})

print ("결과:")
print(result["result"])

print("출처:")
for doc in result["source_documents"]:
    print(doc.page_content)
    print("---")

결과:
문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]

《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]
    질문: 심사위원을 누가 맡았어?
    답변: 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)는 넷플릭스의 요리 서바이벌 프로그램이다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안성재가 맡았다.
심사위원은 백종원과 안

In [ ]:
docs = vectorstore.as_retriever().get_relevant_documents(question)
docs

In [ ]:
docs = vectorstore.similarity_search(question, k=4)
docs

In [ ]:
# It seems vectorDB loading from embedding model works fine, but seems llm model does not.
# Some Korean llm model seems to work fine in text-generation task, but for Question-Ansering task, we might need another approach.

## 2. Use QA pipeline with vectorstor similarity search

In [32]:
# import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load model and tokenizer
model_name = "yjgwak/klue-bert-base-finetuned-squard-kor-v1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set Q_A pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [33]:
# Example: define question and context 
question = "오늘 날씨 어때?"
context = "오늘의 날씨는 맑고 따뜻한 기온이 유지될 것으로 보입니다."

# model chain
result = qa_pipeline(question=question, context=context)

# Result
print("질문:", question)
print("답변:", result['answer'])

질문: 오늘 날씨 어때?
답변: 맑고 따뜻한 기온이


In [34]:
# search context in VectorStore
def retrieve_context(question, vectorstore):
    docs = vectorstore.similarity_search(question, k=4)
    if docs:
        return " ".join([doc.page_content for doc in docs])
        # return docs[0].page_content  # return first relevant doc
    else:
        return None

# Generate answer based on query and searched context similar to RAG chain
def answer_question_with_context(question, vectorstore):
    context = retrieve_context(question, vectorstore)
    if context:
        result = qa_pipeline(question=question, context=context)
        return result['answer'], context  # return answer and used source doc
    else:
        return "관련 문맥을 찾지 못했습니다.", None

In [35]:
# Example
question = "심사위원을 누가 맡았어?"

answer, used_context = answer_question_with_context(question, vectorstore)

print("질문:", question)
print("답변:", answer)
print("사용된 문맥:", used_context)

질문: 심사위원을 누가 맡았어?
답변: 백종원과 안성재가
사용된 문맥: 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1] 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]


## 3. Use Gemini+RAG

In [99]:
from IPython.display import display
import ipywidgets as widgets



In [ ]:
# It seems the best and simple and cost-free option when OpenAI api cannot be used.

In [103]:
pip install -q langchain langchain-community langchain_huggingface chromadb google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip install google-generativeai

In [104]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.llms import OpenAI
import google.generativeai as genai
import os

In [105]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR-API-KEY"
genai_api_key = "u r api"

In [106]:
genai.configure(api_key=genai_api_key)

In [107]:
# 1. Gemini model
gemini_model = genai.GenerativeModel('gemini-1.5-flash')

# 2. embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [108]:
from langchain.vectorstores import Chroma
# sample docs
docs = [
    Document(page_content="한국어 챗봇은 자연어 처리 기술을 사용하여 사용자와 대화를 나눕니다.", metadata={"source": "doc1"}),
    Document(page_content="인공지능을 활용한 챗봇은 여러 산업에서 사용되고 있습니다.", metadata={"source": "doc2"}),
    Document(page_content="한국어와 영어를 동시에 지원하는 챗봇이 점점 늘어나고 있습니다.", metadata={"source": "doc3"}),
    Document(page_content="챗봇은 고객 서비스를 개선하고 사용자 경험을 향상시키는 데 중요한 역할을 합니다.", metadata={"source": "doc4"})
]

# to avoid collision with previous one
persist_directory = "./new_chroma_db"

vectorstore = Chroma.from_documents(splits, embedding=embedding_model, persist_directory="./chroma_db")

In [109]:
# RAG using prompt
def rag_chatbot(question):
    context_doc = vectorstore.similarity_search(question, k=4)
    #context = context_doc[0].page_content if context_doc else "정보를 찾을 수 없습니다."

    context = " ".join([doc.page_content for doc in context_doc])
    
    prompt = f"Context: {context}\질문: {question}\짧게 대답해줘 :"
    # response = gemini_model(prompt)
    
    response = gemini_model.generate_content(prompt)
    answer = response.candidates[0].content.parts[0].text

    #print("출처 문서:", context)
    return answer

In [110]:
# sample question
question = "어떻게 지내니?"
response = rag_chatbot(question)

print("질문:", question)
print("답변:", response)

질문: 어떻게 지내니?
답변: 잘 지내요.



In [112]:
prompt = f"질문: 질풍가도에 대해 알려줘? \ n대답:"
response = gemini_model.generate_content(prompt)

answer = response.candidates[0].content.parts[0].text
print(answer)

질풍가도(疾風歌道)는 여러 가지 의미로 해석될 수 있는 표현입니다. 문맥에 따라 다음과 같은 의미를 가질 수 있습니다.

* **일반적인 의미:** 맹렬한 기세로 앞으로 나아가는 모습, 급속한 발전이나 성장, 거침없는 행보를 묘사하는 관용적인 표현입니다.  빠르고 역동적인 움직임과 압도적인 기세를 강조합니다.  마치 폭풍처럼 거세게 몰아치는 모습을 연상시키죠.

* **문학/예술 작품 제목으로:** 특정 소설, 영화, 음악 등의 제목으로 사용될 수 있으며,  그 작품의 주제나 내용을 암시합니다.  작품의 내용에 따라 질풍가도라는 제목이 가진 의미는 달라질 수 있습니다.  예를 들어, 주인공의 격렬한 삶이나 급격한 변화를 나타낼 수 있습니다.

* **개인의 삶의 태도로:**  자신의 삶을 적극적으로 개척하고, 목표를 향해 끊임없이 도전하는 정신을 나타낼 수 있습니다.  어려움에도 굴하지 않고 끊임없이 전진하는 강인한 의지를 보여줍니다.


질풍가도에 대한 더 자세한 정보를 얻으시려면, 어떤 맥락에서 이 단어를 접하셨는지 알려주시면 더 정확한 답변을 드릴 수 있습니다.  예를 들어, 특정 작품의 제목으로 사용된 경우 작품의 제목을 알려주시면 좋습니다.

